## Import the needed packages

In [1]:
import psycopg2
import pandas as pd
import time, datetime
import json


## Define a function that load the comtrade json files into a dictionary

In [2]:
def load_comtrade_info():
    comtrade_dictionary={}
    json_data=open('lib/classificationHS.json')
    comtrade_dictionary['comcodes']= pd.DataFrame(json.load(json_data)['results'])
    json_data.close()
    json_data=open('lib/partnerAreas.json')
    comtrade_dictionary['partners']= pd.DataFrame(json.load(json_data)['results'])
    json_data.close()
    json_data=open('lib/reporterAreas.json')
    comtrade_dictionary['reporters']= pd.DataFrame(json.load(json_data)['results'])
    json_data.close()
        
    return comtrade_dictionary

## Define three function which lookup comtrade codes in the comtrade dictionary
I will change the country search code to use the ISO country names

In [3]:
def get_reporter_code(comtrade_dictionary, country):
    '''
    Search the reporter country code
    '''
    return comtrade_dictionary['reporters']['id'][comtrade_dictionary['reporters']['text'].str.contains(country)].iloc[0]
    

In [4]:
def get_partner_code(comtrade_dictionary, country):
    '''
    Search the partner country code
    '''
    return comtrade_dictionary['partners']['id'][comtrade_dictionary['partners']['text'].str.contains(country)].iloc[0]


In [5]:
def get_commodity_code(comtrade_dictionary, commodity):
    '''
    Search the commodity code
    '''
    commodity_code = comtrade_dictionary['comcodes'][comtrade_dictionary['comcodes']['text'].str.contains(commodity)]
    if commodity_code.empty:
        print(commodity + ' not found.')
        return -1
    else:
        #take first one if more than one result
        commodity_code = commodity_code.iloc[0]
    print(commodity_code['text'])
    return commodity_code


## Connect to the database

In [6]:
conn = psycopg2.connect(
                 dbname = "comtrade", # could also be "hmrc"
                 host = "data-science-pgsql-dev-01.c8kuuajkqmsb.eu-west-2.rds.amazonaws.com",
                 user = "trade_read",
                 password = "2fs@9!^43g")

cur = conn.cursor()

## Get the column names and print them out

In [7]:
cur.execute("select COLUMN_NAME, DATA_TYPE, NUMERIC_PRECISION from INFORMATION_SCHEMA.COLUMNS where TABLE_NAME='comtrade'")
column_names=pd.DataFrame(cur.fetchall())
print(column_names)

[('rowid', 'bigint', 64), ('classification', 'text', None), ('year', 'integer', 32), ('period', 'integer', 32), ('period_desc', 'text', None), ('aggregate_level', 'integer', 32), ('is_leaf_code', 'integer', 32), ('trade_flow_code', 'integer', 32), ('trade_flow', 'text', None), ('reporter_code', 'integer', 32), ('reporter', 'text', None), ('reporter_iso', 'text', None), ('partner_code', 'integer', 32), ('partner', 'text', None), ('partner_iso', 'text', None), ('commodity_code', 'text', None), ('commodity', 'text', None), ('qty_unit_code', 'integer', 32), ('qty_unit', 'text', None), ('qty', 'bigint', 64), ('netweight_kg', 'bigint', 64), ('trade_value_usd', 'bigint', 64), ('flag', 'integer', 32), ('source', 'integer', 32), ('inserted', 'timestamp without time zone', None)]


In [14]:
cn= pd.DataFrame(column_names)
print(cn)

                  0                            1     2
0             rowid                       bigint  64.0
1    classification                         text   NaN
2              year                      integer  32.0
3            period                      integer  32.0
4       period_desc                         text   NaN
5   aggregate_level                      integer  32.0
6      is_leaf_code                      integer  32.0
7   trade_flow_code                      integer  32.0
8        trade_flow                         text   NaN
9     reporter_code                      integer  32.0
10         reporter                         text   NaN
11     reporter_iso                         text   NaN
12     partner_code                      integer  32.0
13          partner                         text   NaN
14      partner_iso                         text   NaN
15   commodity_code                         text   NaN
16        commodity                         text   NaN
17    qty_

## Lookup the needed comtrade codes for the SQL request

In [8]:
comtrade_dict = load_comtrade_info()
uk_code = get_reporter_code(comtrade_dict, 'United Kingdom')
brazil_code = get_partner_code(comtrade_dict, 'Brazil')
beef = get_commodity_code(comtrade_dict, 'Meat')['id']

02 - Meat and edible meat offal


## Download the comtrade data and put it into a pandas DataFrame

In [11]:
t0 = time.perf_counter()

cur.execute("SELECT partner, netweight_kg, trade_value_usd, period, commodity_code FROM comtrade WHERE "\
            "partner_code = %s "\
            "AND period  BETWEEN 201401 AND 201612"\
            "AND commodity_code = %s"\
            "AND reporter_code = %s", (brazil_code, beef, uk_code))

exports_imports = pd.DataFrame(cur.fetchall(), columns=['partner', 'netweight_kg', 'trade_value_usd', 'period', 'commodity_code'])

t1 = time.perf_counter()
print('Request took: ' +str(datetime.timedelta(seconds=t1-t0)))

Request took: 0:00:02.995069


In [12]:
print(exports_imports)

   partner netweight_kg  trade_value_usd  period commodity_code
0   Brazil         None         15256598  201401             02
1   Brazil         None         16656425  201402             02
2   Brazil         None         11085225  201403             02
3   Brazil         None         10171527  201404             02
4   Brazil         None         13035358  201405             02
5   Brazil         None            58908  201406             02
6   Brazil         None         17121928  201406             02
7   Brazil         None          4807262  201407             02
8   Brazil         None         18362832  201408             02
9   Brazil         None         21727710  201409             02
10  Brazil         None         14655838  201410             02
11  Brazil         None         15864259  201411             02
12  Brazil         None         19937311  201412             02
13  Brazil         None         13648491  201501             02
14  Brazil         None         15625678